In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
os.listdir('/kaggle/input/weather-dataset-rattle-package')

In [ ]:
df = pd.read_csv('/kaggle/input/weather-dataset-rattle-package/weatherAUS.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(df.isna(),cmap='viridis',cbar=False)

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(df.corr(),annot=True,cmap='viridis')

In [ ]:
feature_with_na = [feature for feature in df.columns if df[feature].isnull().sum()>1] 

In [ ]:
for feature in feature_with_na:
    print(feature,np.round(df[feature].isnull().mean()*100,4), '% missing values')

In [ ]:
df.head()

In [ ]:
df['RainTomorrow'] = df['RainTomorrow'].replace('No',0)

In [ ]:
df['RainTomorrow'] = df['RainTomorrow'].replace('Yes',1)

In [ ]:
df.head()

### Missing Features ar importnat or not

In [ ]:
for feature in feature_with_na:
    data = df.copy()
    data[feature] = np.where(data[feature].isnull(),1,0)
    data.groupby(feature)['RainTomorrow'].median().plot.bar()
    plt.title(feature)
    plt.show()

### How many features are numeric feature

In [ ]:
numeric_feature = [feature for feature in df.columns if df[feature].dtype!='O']

In [ ]:
print('Number of numerical feature : ',len(numeric_feature))
df[numeric_feature].head()

### Find Discrete Variable

In [ ]:
discrete_feature = [feature for feature in numeric_feature if len(df[feature].unique()) < 25]

In [ ]:
print('Discrete Feature count  : {} '.format(len(discrete_feature)))

In [ ]:
discrete_feature

In [ ]:
for feature in discrete_feature:
    data = df.copy()
    data.groupby(feature)['RainTomorrow'].median().plot.bar()
    plt.xlabel(feature)
    plt.ylabel('RainTomorrow')
    plt.title(feature)
    plt.show()

In [ ]:
contineous_feature = [feature for feature in numeric_feature if feature not in discrete_feature]

In [ ]:
print('Count of contineous feature are {}'.format(len(contineous_feature)))

In [ ]:
contineous_feature

In [ ]:
for feature in contineous_feature:
    data = df.copy()
    data.groupby(feature)['RainTomorrow'].median().plot.bar()
    plt.xlabel(feature)
    plt.ylabel('RainTomorrow')
    plt.title(feature)
    plt.show()

In [ ]:
for feature in contineous_feature:
    data = df.copy()
    if 0 in data[feature].unique():
        pass
    else:
        data[feature]=np.log(data[feature])
        #data[feature]=np.log(data['RainTomorrow'])
        plt.scatter(data[feature],data['RainTomorrow'])
        plt.xlabel(feature)
        plt.ylabel('RainTomorrow')
        plt.title(feature)
        plt.show()

In [ ]:
df = df.drop('Date',axis=1)

### Get all Categorical Features

In [ ]:
categorical_feature = [feature for feature in df.columns if df[feature].dtype=='O']

In [ ]:
print(categorical_feature)

### Fill all missing values with a new label 'Missing'

In [ ]:
def replace_cat_nan(dataset,feature_nan):
    data=df.copy()
    data[feature_nan] = data[feature_nan].fillna('Missing')
    return data

In [ ]:
df = replace_cat_nan(df,categorical_feature)

In [ ]:
df[categorical_feature].isnull().sum()

In [ ]:
df[categorical_feature].isnull().sum()

### Fill all numerical value with Median

In [ ]:
for feature in numeric_feature:
    median_value = df[feature].median()
    #df[feature + "Nan"] = np.where(df[feature].isnull()1,0)
    df[feature].fillna(median_value,inplace=True)
    
df[numeric_feature].isnull().sum()

In [ ]:
sns.heatmap(df.isnull(),cbar=False,cmap='viridis')

In [ ]:
feature_with_na_values = [feature for feature in df.columns if df[feature].isnull().sum()>1] 

In [ ]:
for feature in feature_with_na_values:
    print(feature,np.round(df[feature].isnull().mean()*100,4), '% missing values')

In [ ]:
print(feature_with_na_values)

In [ ]:
for feature in categorical_feature:
    labels_ordered = df.groupby([feature])['RainTomorrow'].mean().sort_values().index
    labels_ordered = {k:i for i,k in enumerate(labels_ordered,0)}
    df[feature] = df[feature].map(labels_ordered)

In [ ]:
X = df.drop('RainTomorrow',axis=1)
y = df['RainTomorrow']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso

In [ ]:
feature_for_model = SelectFromModel(Lasso(alpha=0.05,random_state=0))
feature_for_model.fit(X_train,y_train)

In [ ]:
feature_for_model.get_support()

In [ ]:
X.columns

In [ ]:
X = df[['Location','Rainfall','Sunshine','WindGustSpeed','WindDir3pm','WindSpeed3pm','Humidity3pm','Pressure3pm']]

In [ ]:
y = df['RainTomorrow']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
print(confusion_matrix(y_test,y_pred))